In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict
from environment import Env

class SARSAgent:
    def __init__(self, env, actions):
        self.env = env
        self.actions = actions
        self.learning_rate = 0.01
        self.discount_factor = 0.9
        self.epsilon = 0.1
        self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0])
        self.rewards_per_episode = []

    def learn(self, state, action, reward, next_state, next_action):
        current_q = self.q_table[state][action]
        next_state_q = self.q_table[next_state][next_action]
        new_q = (current_q + self.learning_rate *
                (reward + self.discount_factor * next_state_q - current_q))
        self.q_table[state][action] = new_q

    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.actions)
        else:
            state_action = self.q_table[state]
            action = self.arg_max(state_action)
        return action

    @staticmethod
    def arg_max(state_action):
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

    def run_episodes(self, env, num_episodes):
        env = env
        success_count = 0
        success_steps = []
        
        for episode in range(num_episodes):
            state = env.reset()
            action = self.get_action(str(state))
            total_reward = 0
            step_count = 0

            while True:
                next_state, reward, done = env.step(action)
                next_action = self.get_action(str(next_state))
                self.learn(str(state), action, reward, str(next_state), next_action)
                total_reward += reward
                state = next_state
                action = next_action

                if done:
                    self.rewards_per_episode.append(total_reward)
                    if total_reward > 0:
                        success_count += 1
                        success_steps.append(step_count)
                    print(f"Episode: {episode + 1}, Reward: {total_reward}")
                    break
                
                step_count += 1

        success_rate = success_count / num_episodes * 100
        print(f"Percentage of successful episodes: {success_rate}%")
        self.plot_rewards_per_episode()
        self.plot_success_steps(success_steps)

    def plot_rewards_per_episode(self):
        plt.plot(self.rewards_per_episode)
        plt.title('Rewards per Episode')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.show()

    @staticmethod
    def plot_success_steps(success_steps):
        plt.plot(success_steps)
        plt.title('Steps Required for Successful Episodes')
        plt.xlabel('Episode')
        plt.ylabel('Steps')
        plt.show()


if __name__ == "__main__":
    env = Env()
    agent = SARSAgent(env, actions=list(range(env.n_actions)))
    agent.run_episodes(env, 100)
